In [34]:
import pandas as pd
import requests
from sqlalchemy import create_engine

In [56]:
class ETL:
    def __init__(self, db_name, user='postgres', host='localhost'):
        self.user = user
        self.host = host
        self.db_name = db_name
        self.engine = create_engine(
            'postgresql://{user}@{host}:5432/{db_name}'.format(user=user, host=host, db_name=db_name)
        )

    def write_to_db(self, path, table_name):
        extension = path.split('.')[-1]
        if extension == 'csv':
            data = pd.read_csv(path, encoding="ISO-8859-1", header=2)
        elif extension == 'xlsx':
            data = pd.read_excel(path, sheetname=2)
        data.to_sql(table_name, self.engine, if_exists='append')
    
    def get_table(self, table_name):
        return pd.read_sql_table(table_name, self.engine)
    
    def get_cb_data(self, url, path):
        '''
        XML is a tree-like structure, while a Pandas DataFrame
        is a 2D table-like structure. So there is no automatic way
        to convert between the two. You have to understand the XML
        structure and know how you want to map its data onto a 2D table,
        so now, when we don't know which xml is needed we would return it
        as text and save to file
        '''
        
        r = requests.get(url)
        with open(path, 'w') as file:
            file.write(r.text)
            
        return r.text

        

In [59]:
etl = ETL(db_name='opentrm')
etl.write_to_db('rb_e20161027.txt.csv', 'rb')
etl.write_to_db('Kospi Quotes Eikon Loader.xlsx', 'kospi')
csv_data = etl.get_cb_data('http://www.cbr.ru/scripts/XML_daily_eng.asp?date_req=02/03/2002', 'cb.xml')
etl.get_table('rb').head()

,index,Issue Code,Issue Name,Put^Call,Contract Month,Strike Price,Settlement Price,Theoretical Price,Underlying Index,Volatility,Interest Rate,Days until Maturity,Underlying Name
0,0,161120018,FUT_NK225_1612,None,201612,NaN,17350.0,17337.0,17336.42,NaN,0.0300,43,Nikkei 225
1,1,162030018,FUT_NK225_1703,None,201703,NaN,17310.0,17303.0,17336.42,NaN,0.1064,134,Nikkei 225
2,2,162060018,FUT_NK225_1706,None,201706,NaN,17170.0,17181.0,17336.42,NaN,0.1282,225,Nikkei 225
3,3,162090018,FUT_NK225_1709,None,201709,NaN,17130.0,17151.0,17336.42,NaN,0.1282,316,Nikkei 225
4,4,162120018,FUT_NK225_1712,None,201712,NaN,17000.0,17044.0,17336.42,NaN,0.1282,407,Nikkei 225


In [60]:
etl.get_table('kospi').head()

,index,DATE1,YEAR,MONTH,EXPIR,STRIKE,cRIC,pRIC,cBID_RH,cBID_TS,...,pASK_RH,pASK_TS,pASK_V,pSTL_RH,pSTL_TS,pSTL_V,fRIC,fSTL_RH,fSTL_TS,fSTL_V
0,0,2016-10-27,2017,10,NaN,110.0,KS200110J7.KS,KS200110V7.KS,Invalid RIC(s): KS200110J7.KS,NaT,...,Invalid RIC(s): KS200110V7.KS,NaT,NaN,Invalid RIC(s): KS200110V7.KS,None,NaN,KSV7,Invalid RIC(s): KSV7,NaT,NaN
1,1,2016-10-27,2017,10,NaN,112.5,KS200112J7.KS,KS200112V7.KS,Invalid RIC(s): KS200112J7.KS,NaT,...,Invalid RIC(s): KS200112V7.KS,NaT,NaN,Invalid RIC(s): KS200112V7.KS,None,NaN,KSV7,Invalid RIC(s): KSV7,NaT,NaN
2,2,2016-10-27,2017,10,NaN,115.0,KS200115J7.KS,KS200115V7.KS,Invalid RIC(s): KS200115J7.KS,NaT,...,Invalid RIC(s): KS200115V7.KS,NaT,NaN,Invalid RIC(s): KS200115V7.KS,None,NaN,KSV7,Invalid RIC(s): KSV7,NaT,NaN
3,3,2016-10-27,2017,10,NaN,117.5,KS200117J7.KS,KS200117V7.KS,Invalid RIC(s): KS200117J7.KS,NaT,...,Invalid RIC(s): KS200117V7.KS,NaT,NaN,Invalid RIC(s): KS200117V7.KS,None,NaN,KSV7,Invalid RIC(s): KSV7,NaT,NaN
4,4,2016-10-27,2017,10,NaN,120.0,KS200120J7.KS,KS200120V7.KS,Invalid RIC(s): KS200120J7.KS,NaT,...,Invalid RIC(s): KS200120V7.KS,NaT,NaN,Invalid RIC(s): KS200120V7.KS,None,NaN,KSV7,Invalid RIC(s): KSV7,NaT,NaN


In [65]:
# database tables schemes
print(pd.io.sql.get_schema(etl.get_table('rb'), 'rb'))

CREATE TABLE "rb" (
"index" INTEGER,
  "Issue Code" INTEGER,
  "Issue Name" TEXT,
  "Put^Call" TEXT,
  "Contract Month" INTEGER,
  "Strike Price" REAL,
  "Settlement Price" REAL,
  "Theoretical Price" REAL,
  "Underlying Index" REAL,
  "Volatility " REAL,
  "Interest Rate" REAL,
  "Days until Maturity" INTEGER,
  "Underlying Name" TEXT
)


/Users/sharif/venvs/machine-learning/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  if __name__ == '__main__':


In [66]:
print(pd.io.sql.get_schema(etl.get_table('kospi'), 'kospi'))

CREATE TABLE "kospi" (
"index" INTEGER,
  "DATE1" TIMESTAMP,
  "YEAR" INTEGER,
  "MONTH" INTEGER,
  "EXPIR" REAL,
  "STRIKE" REAL,
  "cRIC" TEXT,
  "pRIC" TEXT,
  "cBID_RH" TEXT,
  "cBID_TS" TIMESTAMP,
  "cBID_V" REAL,
  "cASK_RH" TEXT,
  "cASK_TS" TIMESTAMP,
  "cASK_V" REAL,
  "cSTL_RH" TEXT,
  "cSTL_TS" TEXT,
  "cSTL_V" REAL,
  "pBID_RH" TEXT,
  "pBID_TS" TIMESTAMP,
  "pBID_V" REAL,
  "pASK_RH" TEXT,
  "pASK_TS" TIMESTAMP,
  "pASK_V" REAL,
  "pSTL_RH" TEXT,
  "pSTL_TS" TEXT,
  "pSTL_V" REAL,
  "fRIC" TEXT,
  "fSTL_RH" TEXT,
  "fSTL_TS" TIMESTAMP,
  "fSTL_V" REAL
)
